# Uses interaction to push triples to the brain

Before running, start GraphDB and make sure that there is a sandbox repository

In [1]:
import json
import os
import time
import uuid
from datetime import date
from datetime import datetime
from random import getrandbits, choice
import pathlib

# general imports for EMISSOR and the BRAIN
import emissor as em
import requests
from cltl import brain
from cltl.brain.long_term_memory import LongTermMemory
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.reply_generation.data.sentences import GREETING, ASK_NAME, ELOQUENCE, TALK_TO_ME
from cltl.reply_generation.lenka_replier import LenkaReplier
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario

/Users/piek/PycharmProjects/cltl-chatbots/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Importing the chatbot utility functions

In [2]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import chatbots.util.driver_util as d_util
import chatbots.util.capsule_util as c_util
import chatbots.intentions.talk as talk
import chatbots.intentions.get_to_know_you as friend

## Standard initialisation of a scenario

In [3]:
from random import getrandbits
import requests
##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = "../../data"

### Define the folder where the images and rdf are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"

### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario = scenarioStorage.create_scenario(scenario_id, datetime.now().microsecond, datetime.now().microsecond, AGENT)

Directory  ../../data/2021-11-04-17:07:20  Created 
Directory  ../../data/2021-11-04-17:07:20/image  Created 


## Specifying the BRAIN

In [4]:
rdffolder = scenario_path + "/" + scenario_id + "/" + "rdf"
log_path = pathlib.Path(rdffolder)
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                                log_dir=log_path,
                                clear_all=True)
replier = LenkaReplier()

2021-11-04 17:07:25,133 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-11-04 17:07:27,975 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents
2021-11-04 17:07:27,976 -     INFO -   cltl.reply_generation.api.LenkaReplier - Booted


In [ ]:
#### Small sequence to learn name of speaker
initial_prompt = f"{choice(GREETING)} {choice(ASK_NAME)} {HUMAN_NAME}?"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

#### Get name from person 
HUMAN_NAME, HUMAN_ID = friend.get_a_name_and_id(scenario, AGENT)

chat = Chat(HUMAN_ID)

#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = f"{choice(TALK_TO_ME)}"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

utterance = ""
#### Get input and loop
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    ###### Getting the next input signals
    utterance = input('\n')
    print(HUMAN_NAME + ": " + utterance)
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    #### Process input and generate reply
    reply = talk.process_text_and_reply(scenario, place_id, location, HUMAN_ID, textSignal, chat, replier, my_brain)
    print(AGENT + ": " + reply)
    textSignal = d_util.create_text_signal(scenario, reply)
    scenario.append_signal(textSignal)


Leolani2: What's up? I would like to know your name! Stranger?



 Piek


Piek
Leolani2: So your name is Piek?



 yes


2021-11-04 17:07:36,535 -     INFO - cltl.triple_extraction.api.Chat (Piek_t_535075)     000 - << Start of Chat with Piek_t_535075 >>
Leolani2: Do you have any gossip?


...

Piek: Who are your friends
2021-11-04 17:16:18,774 -     INFO - cltl.triple_extraction.api.Chat (Piek_t_535075)     009 - Piek_t_535075: "Who are your friends"
UtteranceType.QUESTION
{'predicate': {'label': 'be', 'type': ['verb.stative']}, 'subject': {'label': 'leolani-friends', 'type': ['agent', 'noun.person']}, 'object': {'label': '', 'type': []}}
2021-11-04 17:16:18,781 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: leolani-friends_be_? [agent or person_->_])
Leolani2: I don't know



 Selene likes pizza


Piek: Selene likes pizza
2021-11-04 17:22:40,880 -     INFO - cltl.triple_extraction.api.Chat (Piek_t_535075)     010 - Piek_t_535075: "Selene likes pizza"
UtteranceType.STATEMENT
{'subject': {'label': 'selene', 'type': ['noun.person']}, 'predicate': {'label': 'like', 'type': ['prep']}, 'object': {'label': 'pizza', 'type': ['noun.food']}}
2021-11-04 17:22:41,002 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: selene_like_pizza [person_->_food])
2021-11-04 17:22:41,089 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: new subject - existing object 
Leolani2: I am glad to have learned something new. I had never heard about selene before!



 Who likes pizza


Piek: Who likes pizza
2021-11-04 17:23:07,288 -     INFO - cltl.triple_extraction.api.Chat (Piek_t_535075)     011 - Piek_t_535075: "Who likes pizza"
UtteranceType.QUESTION
{'predicate': {'label': 'like', 'type': ['prep']}, 'subject': {'label': 'pizza', 'type': ['noun.food']}, 'object': {'label': '', 'type': []}}
2021-11-04 17:23:07,291 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: pizza_like_? [food_->_])
Leolani2: I wouldn't know!


In [8]:
#scenario.scenario.end = datetime.now().microsecond
scenarioStorage.save_scenario(scenario)

## End of notebook